In [1]:
import tqdm
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForMaskedLM, AutoTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/open-problems-single-cell-perturbations/multiome_train.parquet
/kaggle/input/open-problems-single-cell-perturbations/multiome_obs_meta.csv
/kaggle/input/open-problems-single-cell-perturbations/sample_submission.csv
/kaggle/input/open-problems-single-cell-perturbations/adata_train.parquet
/kaggle/input/open-problems-single-cell-perturbations/multiome_var_meta.csv
/kaggle/input/open-problems-single-cell-perturbations/adata_obs_meta.csv
/kaggle/input/open-problems-single-cell-perturbations/id_map.csv
/kaggle/input/open-problems-single-cell-perturbations/de_train.parquet
/kaggle/input/open-problems-single-cell-perturbations/adata_excluded_ids.csv


In [2]:
de_train = pd.read_parquet('../input/open-problems-single-cell-perturbations/de_train.parquet')
id_map = pd.read_csv('../input/open-problems-single-cell-perturbations/id_map.csv')
sample_submission = pd.read_csv('../input/open-problems-single-cell-perturbations/sample_submission.csv', index_col='id')

### Distributional Analysis

In [ ]:
de_train.iloc[:,5:].describe()

## Data Creation


In [ ]:
de_train=de_train.sample(frac=1)
thr=int(de_train.shape[0]*0.8)
train_data=de_train.iloc[:thr]
test_data=de_train.iloc[thr:]

In [ ]:
# train_data.sm_name.unique()

In [ ]:
chemberta = AutoModelForMaskedLM.from_pretrained("DeepChem/ChemBERTa-77M-MTR")
tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MTR")
chemberta._modules["lm_head"] = nn.Identity()

In [ ]:
def chemberta_features(smiles):
    encoded_input=tokenizer(smiles,return_tensors="pt", padding=False, truncation=True)
    model_output = chemberta(**encoded_input)
    return model_output.logits.mean(dim=1)

In [ ]:
gene_keys=list(de_train.keys()[5:])
cell_gene_mean = train_data[['cell_type']+gene_keys].groupby('cell_type').mean().reset_index()
sm_gene_mean = train_data[['sm_name']+gene_keys].groupby('sm_name').mean().reset_index()
cell_gene_std = train_data[['cell_type']+gene_keys].groupby('cell_type').std().reset_index().replace({np.nan:0})
sm_gene_std = train_data[['sm_name']+gene_keys].groupby('sm_name').std().reset_index().replace({np.nan:0})

In [ ]:
cell_type_len=len(train_data.cell_type.unique())
sm_name_len=len(train_data.sm_name.unique())

In [ ]:
cell_type_dict={item:i for i,item in enumerate(train_data.cell_type.unique())}
sm_dict={item:i for i,item in enumerate(train_data.sm_name.unique())}

In [56]:
class Drug_Dataset(Dataset):
    def __init__(self, df,features,):
        self.df = df
        self.features=features
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row=self.df.iloc[idx]
        x=np.array([])
        y=torch.tensor(np.array(row[gene_keys]).astype(np.float32),dtype=torch.float32)
        
        for f in self.features:
            indicator=f.keys()[0]
            ind=row[indicator]
            extracted_f=f[f[indicator]==ind].iloc[0,list(range(1,f.shape[1]))]
            x=np.concatenate([x,np.array(extracted_f).astype(np.float32)],axis=0)
        x=np.concatenate([x,chemberta_features(row['SMILES']).detach().numpy()[0,:]],axis=0)
        x=torch.tensor(x,dtype=torch.float32)
        cell=torch.tensor(cell_type_dict[row['cell_type']])
        drug=torch.tensor(sm_dict[row['sm_name']])
        return {'x':x,'cell':cell,'drug':drug,'y':y}


In [57]:
train_p=Drug_Dataset(train_data,[cell_gene_mean,sm_gene_mean,cell_gene_std,sm_gene_std])
test_p=Drug_Dataset(test_data,[cell_gene_mean,sm_gene_mean,cell_gene_std,sm_gene_std])

In [59]:
train_loader=DataLoader(train_p,batch_size=32,shuffle=True)
test_loader=DataLoader(test_p,batch_size=8)

In [60]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [61]:
class Decoder(torch.nn.Module):
    def __init__(self, embed_dim=16,dim1=64,dim2=128):
        super(Decoder, self).__init__()
        self.cell_type_embeddings = nn.Embedding(cell_type_len, embed_dim)
        self.sm_name_embeddings = nn.Embedding(sm_name_len, embed_dim)
#         self.bn0=nn.BatchNorm1d(embed_dim*2+train_p[0]['x'].shape[0])
        self.dense1=nn.Linear(embed_dim*2+train_p[0]['x'].shape[0],dim1)
        self.bn1=nn.BatchNorm1d(dim1)
        self.dp1=nn.Dropout(0.2)
        self.dense2=nn.Linear(dim1,dim2)
        self.bn2=nn.BatchNorm1d(dim2)
        self.dp2=nn.Dropout(0.2)
        self.dense3=nn.Linear(dim2,18211)

    def forward(self, x_cell,x_sm,x):
        x1 = self.cell_type_embeddings(x_cell)
        x2 = self.sm_name_embeddings(x_sm)
        x=torch.concat([x1,x2,x],dim=-1)
#         x=self.bn0(x)
        x=self.dp1(F.relu((self.bn1(self.dense1(x)))))
        x=self.dp2(F.relu((self.bn2(self.dense2(x)))))
        x=self.dense3(x)
        return x


In [62]:
model=Decoder(embed_dim=16,dim1=64,dim2=128)
model.to(device)

Decoder(
  (cell_type_embeddings): Embedding(6, 16)
  (sm_name_embeddings): Embedding(146, 16)
  (dense1): Linear(in_features=73260, out_features=64, bias=True)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dp1): Dropout(p=0.2, inplace=False)
  (dense2): Linear(in_features=64, out_features=128, bias=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dp2): Dropout(p=0.2, inplace=False)
  (dense3): Linear(in_features=128, out_features=18211, bias=True)
)

In [63]:
def mrrmse(outputs,y):
    return (torch.sqrt(((outputs-y)**2).mean(dim=-1))).mean()

In [64]:
def train(num_epochs=1000):
    criterion= nn.MSELoss()
    criterion2= mrrmse
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    best_test_loss = float('inf')

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_loss2 = 0.0
        train_total=0.0
        test_loss=0.0
        test_loss2=0.0
        test_total=0.0
        

        for batch in tqdm.tqdm(train_loader):
            cell=batch['cell'].to(device)
            drug=batch['cell'].to(device)
            x=batch['x'].to(device)
            y=batch['y'].to(device)
            optimizer.zero_grad()
            outputs = model(cell,drug,x)
            loss = criterion(outputs,y)
            loss2 = criterion2(outputs,y)
            train_loss += loss.item()
            train_total+=outputs.size(0)
            train_loss2 += loss2.item()
            loss.backward()
            optimizer.step()
        avg_train_loss = train_loss / len(train_loader)
        avg_train_loss2 = train_loss2 / len(train_loader)


        model.eval()
        with torch.no_grad():
            for batch in test_loader:
                cell=batch['cell'].to(device)
                drug=batch['cell'].to(device)
                x=batch['x'].to(device)
                y=batch['y'].to(device)
                outputs = model(cell,drug,x)
                loss1 = criterion(outputs,y)
                loss2 = criterion2(outputs,y)
                test_total+=outputs.size(0)
                test_loss += loss.item()
                test_loss2 += loss2.item()
        avg_test_loss = test_loss / len(test_loader)
        avg_test_loss2 = test_loss2 / len(test_loader)

        if avg_test_loss < best_test_loss:
            best_test_loss = avg_test_loss
            torch.save(model.state_dict(), "best_model.pt")
        print(f"Epoch {epoch+1}/{num_epochs}: "
              f"Train MSE Loss: {avg_train_loss:.4f} , Train MRRMSE Loss: {avg_train_loss2:.4f}, Valid MSE Loss: {avg_test_loss:.4f}, Valid MRRMSE Loss: {avg_test_loss2:.4f}")

In [ ]:
train()

100%|██████████| 16/16 [01:08<00:00,  4.31s/it]


Epoch 1/1000: Train MSE Loss: 5.4611 , Train MRRMSE Loss: 1.3457, Valid MSE Loss: 1.2459, Valid MRRMSE Loss: 1.2358


100%|██████████| 16/16 [01:09<00:00,  4.31s/it]


Epoch 2/1000: Train MSE Loss: 5.5421 , Train MRRMSE Loss: 1.3541, Valid MSE Loss: 5.5180, Valid MRRMSE Loss: 1.2240


100%|██████████| 16/16 [01:08<00:00,  4.31s/it]


...

100%|██████████| 16/16 [01:09<00:00,  4.35s/it]


Epoch 140/1000: Train MSE Loss: 2.4939 , Train MRRMSE Loss: 1.0886, Valid MSE Loss: 0.9581, Valid MRRMSE Loss: 1.2264


100%|██████████| 16/16 [01:09<00:00,  4.34s/it]


Epoch 141/1000: Train MSE Loss: 2.4063 , Train MRRMSE Loss: 1.0686, Valid MSE Loss: 1.7909, Valid MRRMSE Loss: 1.2446


 62%|██████▎   | 10/16 [00:45<00:27,  4.51s/it]

In [55]:
for batch in tqdm.tqdm(train_loader):
    cell=batch['cell'].to(device)
    drug=batch['cell'].to(device)
    y=batch['y'].to(device)
    outputs = model(cell,drug)
    loss = nn.L1Loss()(outputs,y)
    break

  0%|          | 0/16 [00:02<?, ?it/s]


In [56]:
loss

tensor(1.1626, grad_fn=<MeanBackward0>)

In [57]:
y

tensor([[-0.7790, -0.3305, -1.1666,  ...,  0.7030,  0.8946,  0.2529],
        [ 0.4641,  0.1390,  0.6811,  ...,  0.5811,  0.4223, -0.7910],
        [ 0.0288, -0.7789, -3.6663,  ...,  1.1905, -3.5731,  0.8906],
        ...,
        [ 0.1066,  0.0069,  0.4245,  ..., -0.6688,  0.7983, -0.0600],
        [ 0.6422, -0.4220,  0.4593,  ...,  0.0229, -0.0209,  0.4794],
        [ 2.2251,  0.5643, -0.4376,  ...,  3.3962, -0.5671,  0.5014]])

In [58]:
outputs

tensor([[ 6.2332e-02, -2.1571e-01,  2.4501e-02,  ...,  6.1414e-02,
          8.4924e-02, -1.6814e-01],
        [ 3.8393e-01, -4.5674e-01, -5.3927e-01,  ..., -4.0812e-02,
         -2.8494e-01,  1.1883e-03],
        [ 7.3435e-02, -1.4179e-01, -2.9638e-01,  ...,  3.4764e-01,
         -1.1420e-01, -1.3031e-01],
        ...,
        [ 4.1656e-02,  4.7643e-01, -4.7299e-01,  ..., -6.7893e-01,
         -1.8814e-01, -1.7352e-01],
        [-3.7704e-01, -1.2595e-02, -1.6405e-01,  ..., -1.5800e-01,
         -2.3328e-01, -1.0147e-01],
        [ 1.2016e+00,  1.8736e-01,  4.0568e-01,  ...,  5.7204e-01,
         -4.1059e-01,  8.9473e-01]], grad_fn=<AddmmBackward0>)